## import libraries

In [8]:
from pytube import YouTube
import cv2
import numpy as np
from PIL import Image as im 
from sklearn.neighbors import LocalOutlierFactor

download_dir = "youtube-download"
slide_dir = "youtube-slide"

## Take YouTube Video link as input and Download the Video

In [9]:
link = input("Enter the YouTube Video Link Containing Slide: ")
link = link.split("&")[0]
yt = YouTube(link)
try:
    title = yt.title.split()[0] + ".mp4"
except:
    title = "current_video.mp4"

try:
    try:
        yt.streams.filter(adaptive=True, only_video=True).order_by('resolution').desc().first().download(download_dir, title)
    except KeyError as e:
        yt.streams.filter(progressive=True).order_by('resolution').desc().first().download(download_dir, title)
except AttributeError as ae:
    print("Attribute Error try another video")

In [10]:
title

'current_video.mp4'

## Read the YouTube Video and then Extract frame and differ

In [12]:
import cv2
import numpy as np

download_dir = "youtube-download"
slide_dir = "youtube-slide"
cap = cv2.VideoCapture(download_dir + "/" + title)


if (cap.isOpened() == False):
    print("Error opening video file")

list_of_frame = []
previous_Frame = None
threshold = 8.0 ## replace this value
diff_values = []
frame_count = 0
fps = cap.get(cv2.CAP_PROP_FPS)
skip_sec = 2
while(cap.isOpened()):
    frame_id = int(fps*(skip_sec * frame_count))
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_id)
    ret, frame = cap.read()
    if ret:
        if previous_Frame is not None:
            h, w, _ = frame.shape
            diff = cv2.subtract(frame, previous_Frame)
            diff_value  = np.sum(diff) / (h * w)
            # print(diff_value, frame_count, frame_id)
            diff_values.append(diff_value)
        previous_Frame = frame
        list_of_frame.append(frame)
        frame_count += 1 
    else:
        cap.release()
        break
    

In [13]:
diff_values_expand_dims = np.expand_dims(np.array(diff_values), axis=1)
# diff_values_expand_dims

In [14]:
# predicting slides using Anomaly Detection 

lof = LocalOutlierFactor(n_neighbors=20, n_jobs=-1)
prediction = lof.fit_predict(diff_values_expand_dims)
list_of_frame_arr = np.array(list_of_frame)
frames_for_slide = list_of_frame_arr[np.where(prediction==-1)[0]]
images = []
for i in range(frames_for_slide.shape[0]):
    img_arr = frames_for_slide[i]
    img_arr = cv2.cvtColor(img_arr, cv2.COLOR_BGR2RGB)
    images.append(im.fromarray(img_arr))
images[0].save(slide_dir + "/" + title.split('.')[0]+".pdf", save_all=True, append_images = images[1:])


In [17]:
# predicting slides using Thresholding values

list_of_frame_arr = np.array(list_of_frame)
frames_for_slide = list_of_frame_arr[np.where(np.array(diff_values) > 8)[0]]
images = []
for i in range(frames_for_slide.shape[0]):
    img_arr = frames_for_slide[i]
    img_arr = cv2.cvtColor(img_arr, cv2.COLOR_BGR2RGB)
    images.append(im.fromarray(img_arr))
images[0].save(slide_dir + "/" + title.split('.')[0]+"_copy.pdf", save_all=True, append_images = images[1:])
